<h1 align='center'> Применение модели машинного обучения <br>в прогнозировании исхода <br>шахматного матча </h1>


<h3> Скачивание данных и парсинг </h3>
<p align='left'> <strong>Над проектом трудились:</strong>
<br> Парамонов Всеволод
<br> Сидоров Иван
<br> Чубов Артем

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import string

from IPython.display import display, display_html, clear_output
import tqdm

import numpy as np
import pandas as pd
pd.options.display.max_columns=999
from datetime import datetime, timedelta

import zipfile

import seaborn as sns
import matplotlib.pyplot as plt

<h2> Скачивание данных игр с сайта lichess.org за март 2023</h2>

In [ ]:
###########################################
### Преобразование PGN-файла в DataFrame ###
############################################

NUM_GAMES = 3*10**4

rows = []
game_counter = 0
with open("W:\lichess_db_standard_rated_2023-03.pgn\Игры.pgn") as pgn_file:
    for i in tqdm.tqdm(range(NUM_GAMES)):
        game = chess.pgn.read_game(pgn_file)
        game_counter += 1
        if game_counter % 50 == 0:  # извлекаем только каждую 50-ю игру
            row = {}
            row['headers'] = game.headers.__dict__
            rows.append(row)
games = pd.DataFrame(rows)

#### **Пояснение:** здесь мы запускали данных цикл, чтобы считать pgn файл и преобразовать его в pandas dataframe, с целью дальнейшей работы над ним

<h2> Подготовка датафрейма для последующего парсинга признаков о соперниках </h2>

In [ ]:
def fix_json_string(json_string):
    # Замена одинарных кавычек на двойные внутри значений JSON
    pattern = r'\'(.*?)\''
    fixed_json_string = re.sub(pattern, r'"\1"', json_string)
    return fixed_json_string

def fix_json_string_2(json_string):
    # Замена двойных кавычек на одинарные внутри значения "Opening"
    fixed_json_string = json_string.replace('"Opening": "', "'Opening': '").replace('", "', "', '").replace('"}', "'}")
    return fixed_json_string

l = []
for el in games.headers:
    f_1 = fix_json_string_2(el)
    fixed_json_string = fix_json_string(f_1)
    try:
        a = json.loads(fixed_json_string)
    except:
        a = np.nan
    l.append(a)

#### **Пояснение:** здесь происходит преобразования полученного раннее dataframe, а точнее преобразование данных, чтобы с ними можно было работать

#### После обработки данных считываем полученный файл

In [2]:
games_stats = pd.read_csv('/Users/ivansidorov/Downloads/Датасет шахматисты мнение.csv')


<h2> Скрипт для парсинга дополнительных признаков соперников </h2>

In [7]:
from datetime import datetime, timedelta

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:101.0) Gecko/20100101 Firefox/101.0'
}

#####################################################
### Функция по парсингу дополнительных признаков: ###
#####################################################
####                                             ####
###  Статистика матча 1 (до рассматриваемого)     ###
###                                               ###
###  Статистика матча 2 (до рассматриваемого)     ###
###                                               ###
###  Статистика матча 2 (до рассматриваемого)     ###
####                                             ####
#####################################################


def parsing_add_feat(utc_date, utc_time, pl, op_pl):


    ########################################
    ### Форматируем дату проведения игры ###
    ########################################


    DESIRED_MONTH = 9
    DESIRED_YEAR = 2023

    hours = utc_time // 3600
    minutes = (utc_time % 3600) // 60
    seconds = (utc_time % 60)


    desired_datetime_utc = datetime(
    DESIRED_YEAR,
    DESIRED_MONTH,
    utc_date,
    hours,
    minutes,
    seconds
    )
        

    # Преобразуем время в формат "HH:MM:SS"
    time_str = desired_datetime_utc.strftime("%H:%M:%S")

    # Получаем дату в формате "YYYY-MM-DD"
    date_str = desired_datetime_utc.strftime("%Y-%m-%d")

    # Комбинируем дату и время в одну строку
    formatted_datetime = date_str + "T" + time_str


    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:101.0) Gecko/20100101 Firefox/101.0'
    }

    nickname = pl
    current_opponet = op_pl

    ##############################################################
    ### Создаем макет для записывания результатов по 3-м играм ###
    ##############################################################



    data = {'color_game_1' : None,
            'opp_rating_1' : None,
            'rezult_game_1' : None,
            'color_game_2' : None,
            'opp_rating_2' : None,
            'rezult_game_2' : None,
            'color_game_3' : None,
            'opp_rating_3' : None,
            'rezult_game_3' : None
    
    }

    games_parsed = 0

    i = 1

    found_opp = []

    ###################################################
    ### Запускам перелистывание страниц до тех пор, ###
    ### пока не будет спаршено 3х игр               ###    
    ###################################################

    while games_parsed != 3:

        
        url_3 = f'https://lichess.org/@/{nickname}/all?page={i}'

        response = requests.get(url_3, headers=headers)
        gett = BeautifulSoup(response.content, 'html.parser')

        #########################################################
        ### Делаем проверку на то, что все возможные страницы ###
        ### были пролистаны до лимита (40 страниц)            ###
        #########################################################

        try:

            games = gett.find_all('div', {'class': 'angle-content'})[0]

        except:
            break


        games = games.find_all('div', {'class': 'search__result'})[0]

        games = games.find_all('article', {'class': 'game-row paginated'})

        ###############################################
        ### Проходимся по каждому матчу на странице ###
        ###############################################


        for j in games:

            if games_parsed == 3:
                break
            else:

                gm = j.find_all('div', {'class': 'game-row__infos'})[0]

                time = gm.find_all('time', {'class': 'timeago'})[0].get('datetime')

                #####################################################
                ### Проверяем, что текущая игра произошла раньше, ###
                ### чем рассматриваемая                           ###
                #####################################################

                if time < formatted_datetime:

                    vers = gm.find_all('div', {'class': 'versus'})[0]
                    players = vers.find_all('a', {'class': 'user-link ulpt'})

                    ############################################################
                    ### Тут происходит проверка на то, что игра была сыграна ###
                    ### между двумя людьми (может возникнуть такой случай,   ###
                    ### что человек играет с ботом                           ###
                    ############################################################

                    try:
                        white_player, black_player = players

                    except:
                        continue

                    #########################################################
                    ### Делаем проверку на то, что мы рассматриваем игры, ###
                    ### которые произошли до рассматриваемого матча       ### 
                    #########################################################                       

                    if white_player.text == current_opponet or black_player.text == current_opponet:
                        found_opp.append(1)
                        continue
                    
                    if len(found_opp) == 1:

                        color_coded = 0
                        color = ''
                        opp_color = ''

                        ###############################################################
                        ### Определяем цвет, за который играл рассматриваемый игрок ###
                        ###############################################################

                        if white_player.text == nickname:
                            color = 'white'
                            color_coded = 1
                            opp_color = 'black'

                        else:
                            color = 'black'
                            color_coded = 0
                            opp_color = 'white'

                        data[f'color_game_{games_parsed+1}'] = color_coded

                        ####################################################
                        ### Определяем результат игры, на котором сейчас ###
                        ### находится парсер                             ###    
                        ####################################################                

                        rez_1 = gm.find_all('span', {'class': 'loss'})
                        rez_2 = gm.find_all('span', {'class': 'win'})

                        verd = ''

                        if len(rez_1) != 0:
                            verd = rez_1[0].text
                        elif len(rez_2) != 0: verd = rez_2[0].text
                        else: verd = 'Stalemate'

                        if 'Black is victorious' in verd:

                            data[f'rezult_game_{games_parsed+1}'] = 0

                        else: 
                            data[f'rezult_game_{games_parsed+1}'] = 1

                        ##################################################
                        ### Достаем рейтинг оппонента, с которым играл ###
                        ### рассматриваемый игрок                      ###
                        ##################################################

                        opp_parsing = gm.find_all('div', {'class': f'player {opp_color}'})[0]
                        br_element = opp_parsing.find('br')
                        text_after_br = br_element.next_sibling.strip()

                        data[f'opp_rating_{games_parsed+1}'] = int(text_after_br.replace('?', ''))

                        games_parsed += 1

        i+=1

    df = pd.DataFrame([list(data.values())], columns=list(data.keys()))

    return df

<h2> Скрипт для парсинга признаков с сайта lichess.org </h2>

In [10]:
##############################################
### Функция по парсингу основных признаков ###
##############################################
####                                      ####
### Статистики по разным режимам           ###
###                                        ###
### Результаты за все игры (победы, ничьи  ###
### поражения)                             ###
###                                        ###
### Текущие серии побед и поражений        ###
###                                        ###
### Стандартное отклонение рейтинга        ###
####                                      #### 
##############################################

def get_stats(d):
    pl, match = d

    url = 'https://lichess.org/@/' + pl + '/all'


    ratings = {'Username': pl,
               'Mode': match,
               'UltraBullet': 0,
               'Bullet': 0,
               'Blitz': 0,
               'Rapid': 0,
               'Classical': 0,
               'Correspondence': 0,
               'Crazyhouse': 0,
               'Chess960': 0,
               'King of the Hill': 0,
               'Three-check': 0,
               'Antichess': 0,
               'Atomic': 0,
               'Horde': 0,
               'Racing Kings': 0,
               'Puzzles': 0,
               'Puzzle Storm': 0,
               'Puzzle Racer': 0,
               'Puzzle Streak': 0}
    
    
    games = {'games': 0,
           'rated': 0,
           'wins': 0,
           'losses': 0,
           'draws': 0,
           'bookmarks':0,
           'imported games': 0}
 

    streaks = {'win_streak': 0,
               'lose_streak': 0,
               'deviation': 0
               }    
    

    ################################################
    ### Парсинг данных с боковой страницы игрока ###
    ################################################

    responsse = requests.get(url, headers=headers)
    tre = BeautifulSoup(responsse.content, 'html.parser')
    statistic = tre.find_all('div', {'class': 'side sub-ratings'})

    
    ### Проверка на закрытость профиля игрока ###
    
    try:
        statistic = statistic[0].find_all('span')
    except:
        
        ### Если профиль игрока закрыт, то возвращаем DataFrame с NaN значениями ###
        
        ratings = {x: np.NaN for x in ratings}
        games = {x: np.NaN for x in games}
        streaks = {x: np.NaN for x in streaks}
        df1 = pd.DataFrame([list(ratings.values())], columns=list(ratings.keys()))
        df2 = pd.DataFrame([list(games.values())], columns=list(games.keys()))
        df3 = pd.DataFrame([list(streaks.values())], columns=list(streaks.keys()))

        df = pd.concat([df1, df2, df3], axis=1)
        return df
    
    ### Запись рейтингов и количества игр в словарь ###

    for i in statistic:
        if 'h3' in str(i):
            if i.find_all('strong')[0].text != '?':
                val = i.find_all('strong')[0].text.translate(str.maketrans('', '', string.punctuation))
                ratings[i.find_all('h3')[0].text] = int(val)




    #############################################
    ### Парсинг статистики игрока за все игры ###
    ##      (победы, поражения, ничьи)         ##      
    #############################################

    gms = tre.find_all('div', {'class': 'number-menu number-menu--tabs menu-box-pop'})
    gms = gms[0].find_all('a')
    
    for j in gms:
        res = j.text.translate(str.maketrans('', '', string.punctuation))
        res = re.split('(\d+)', res)[1:]
        points, rezult = res

        ### Решение проблемы со множественным числом ###

        if rezult in ['win', 'game', 'loss', 'draw', 'bookmark']:
            if rezult == 'loss':
                rezult += 'es'
            else: rezult += 's'        
        games[rezult] = int(points)


    ###############################################
    ### Парсинг текущих серий побед и поражений ###
    ###############################################

    match = match.lower()
    if match == 'ultra bullet':
        match = 'ultaBullet'

    url_2 = f'https://lichess.org/@/{pl}/perf/{match}'

    response = requests.get(url_2, headers=headers)
    dop = BeautifulSoup(response.content, 'html.parser')
    
    ### Проверка на то, есть ли у игрока текущие серии ###

    try:
        streak = dop.find_all('section', {'class': 'resultStreak split'})[0]
        streaks['win_streak'] = streak.find_all('strong')[1].text
        streaks['lose_streak'] = streak.find_all('strong')[2].text
    except:

        ### Иначе оставляем 0 ###

        pass

    ################################################
    ### Парсинг стандартного отклонения рейтинга ###
    ################################################
    
    try:
        dev = dop.find_all('section', {'class': 'glicko'})[0]
        a = dev.find_all('strong', {'title': 'Lower value means the rating is more stable. Above 110, the rating is considered provisional. To be included in the rankings, this value should be below 75 (standard chess) or 65 (variants).'})[0].text
        streaks['deviation'] = float(a)
    except:
        a = np.nan
        streaks['deviation'] = float(a)

    ############################################
    ### Преобразование словарей в Data Frame ###
    ############################################

    df1 = pd.DataFrame([list(ratings.values())], columns=list(ratings.keys()))
    df2 = pd.DataFrame([list(games.values())], columns=list(games.keys()))
    df3 = pd.DataFrame([list(streaks.values())], columns=list(streaks.keys()))

    df = pd.concat([df1, df2, df3], axis=1)
    return df


<h2> Разделение датафрейма на игроков, играющих за черные и белые фигуры </h2>

In [8]:
games_stats_white = games_stats[['White', 'Event']].copy()
games_stats_black = games_stats[['Black', 'Event']].copy()
space = "\xa0" * 30

df1_styler = games_stats_white.head(10).style.set_table_attributes("style='display:inline'").set_caption('games_stats_white')
df2_styler = games_stats_black.head(10).style.set_table_attributes("style='display:inline'").set_caption('games_stats_black')

display_html(df1_styler._repr_html_() + space + df2_styler._repr_html_(), raw=True)

,White,Event
0,Apostolis_L,Rated Rapid tournament https://lichess.org/tournament/5xNbZvbs
1,Chawteenn,Rated Blitz game
2,mathmatical,Rated Blitz game
3,Haze0809,Rated Rapid game
4,PaPanDreo,Rated Bullet game
5,yugo28,Rated Blitz game
6,remarkableemperor991,Rated Blitz game
7,HerrNilsson,Rated Bullet game
8,Goldenzmith,Rated Bullet game
9,piedrasanta,Rated Blitz game


<h2> Применение скрипта к игрокам, играющим за белые фигуры </h2>

In [21]:
sample = pd.DataFrame(columns=['Username','Mode','UltraBullet', 'Bullet', 'Blitz', 'Rapid', 'Classical',
       'Correspondence', 'Crazyhouse', 'Chess960', 'Antichess', 'Horde',
       'Puzzles', 'Puzzle Storm', 'Puzzle Racer', 'Puzzle Streak', 'games',
       'rated', 'wins', 'losses', 'draws', 'bookmarks', 'win_streak',
       'lose_streak'])

whites = sample.copy()

#################################################################################################
### Применяем функцию до добавлению признаков к исходным данным по играм для игроков за белых ###
#################################################################################################

for i in games_stats_white.values:
    getting = get_stats(i)
    whites = pd.concat([whites, getting])

whites = whites.reset_index(drop=True)
whites

,Username,Mode,UltraBullet,Bullet,Blitz,Rapid,Classical,Correspondence,Crazyhouse,Chess960,Antichess,Horde,Puzzles,Puzzle Storm,Puzzle Racer,Puzzle Streak,games,rated,wins,losses,draws,bookmarks,win_streak,lose_streak,King of the Hill,Three-check,Atomic,Racing Kings,imported games,deviation,imported game,playing
0,Apostolis_L,Rated Rapid tournament https://lichess.org/tou...,0,0,1034,1337,0,0,0,0,0,0,1421,15,0,11,385,372,103,274,8,0,0,0,0.0,0.0,0.0,0.0,109.0,NaN,NaN,NaN
0,Chawteenn,Rated Blitz game,0,1733,1850,0,0,0,0,0,0,0,1432,0,0,0,15065,15044,7171,7023,871,17,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,mathmatical,Rated Blitz game,0,2037,1978,2072,0,2275,0,0,0,0,2415,0,0,0,20147,20073,10115,9364,668,0,0,0,0.0,0.0,0.0,0.0,6.0,NaN,NaN,NaN
0,Haze0809,Rated Rapid game,0,1027,944,1266,1201,0,0,0,0,0,1486,0,0,0,1322,1294,624,653,45,2,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,PaPanDreo,Rated Bullet game,0,1554,1632,1820,1870,0,0,0,0,0,0,0,0,0,28568,28557,13732,13806,1030,1,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Ngaytaovsmay,Rated Blitz game,0,1541,996,0,0,0,0,0,0,0,1488,0,0,0,362,337,167,184,11,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,bluepenquin,Rated Bullet game,1061,1554,1343,0,0,0,0,0,0,0,2328,0,0,10,4471,3903,2134,2193,144,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,GreySharkfanclubbot,Rated Blitz game,1744,2098,2333,2334,0,0,0,0,0,0,0,0,0,0,12444,12333,5931,5060,1453,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,Hughiez,Rated Blitz game,0,681,937,1188,0,0,0,0,0,0,1342,11,0,9,6652,6525,3103,3468,81,3,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [15]:
whites

,Username,Mode,UltraBullet,Bullet,Blitz,Rapid,Classical,Correspondence,Crazyhouse,Chess960,Antichess,Horde,Puzzles,Puzzle Storm,Puzzle Racer,Puzzle Streak,games,rated,wins,losses,draws,bookmarks,win_streak,lose_streak,King of the Hill,Three-check,Atomic,Racing Kings,imported games,deviation,imported game,playing
0,Apostolis_L,Rated Rapid tournament https://lichess.org/tou...,0,0,1034,1337,0,0,0,0,0,0,1421,15,0,11,385,372,103,274,8,0,0,0,0.0,0.0,0.0,0.0,109.0,NaN,NaN,NaN
0,Chawteenn,Rated Blitz game,0,1733,1850,0,0,0,0,0,0,0,1432,0,0,0,15065,15044,7171,7023,871,17,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,mathmatical,Rated Blitz game,0,2037,1978,2072,0,2275,0,0,0,0,2415,0,0,0,20147,20073,10115,9364,668,0,0,0,0.0,0.0,0.0,0.0,6.0,NaN,NaN,NaN
0,Haze0809,Rated Rapid game,0,1027,944,1266,1201,0,0,0,0,0,1486,0,0,0,1322,1294,624,653,45,2,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,PaPanDreo,Rated Bullet game,0,1554,1632,1820,1870,0,0,0,0,0,0,0,0,0,28568,28557,13732,13806,1030,1,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Ngaytaovsmay,Rated Blitz game,0,1541,996,0,0,0,0,0,0,0,1488,0,0,0,362,337,167,184,11,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,bluepenquin,Rated Bullet game,1061,1554,1343,0,0,0,0,0,0,0,2328,0,0,10,4471,3903,2134,2193,144,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,GreySharkfanclubbot,Rated Blitz game,1744,2098,2333,2334,0,0,0,0,0,0,0,0,0,0,12444,12333,5931,5060,1453,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,Hughiez,Rated Blitz game,0,681,937,1188,0,0,0,0,0,0,1342,11,0,9,6652,6525,3103,3468,81,3,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [17]:
games_stats_black[:10983]

,Black,Event
0,laurent971,Rated Rapid tournament https://lichess.org/tou...
1,EidosRock,Rated Blitz game
2,Ilieck,Rated Blitz game
3,Keykodeus,Rated Rapid game
4,Solus-Christus,Rated Bullet game
...,...,...
10978,wHiterAven34ece,Rated Blitz game
10979,eitery,Rated Bullet game
10980,Leto-II-Atreides,Rated Blitz game
10981,VictorXI,Rated Blitz game


<h2> Применение скрипта к игрокам, играющим за черные фигуры </h2>

In [18]:
blacks = sample.copy()

##################################################################################################
### Применяем функцию до добавлению признаков к исходным данным по играм для игроков за черных ###
##################################################################################################

for i in games_stats_black[:10983].values:
    getting = get_stats(i)
    blacks = pd.concat([blacks, getting])

blacks = blacks.reset_index(drop=True)
blacks

,Username,Mode,UltraBullet,Bullet,Blitz,Rapid,Classical,Correspondence,Crazyhouse,Chess960,Antichess,Horde,Puzzles,Puzzle Storm,Puzzle Racer,Puzzle Streak,games,rated,wins,losses,draws,bookmarks,win_streak,lose_streak,King of the Hill,Three-check,Atomic,Racing Kings,imported games,deviation,imported game,playing
0,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,EidosRock,Rated Blitz game,0,0,1943,0,0,0,0,0,0,0,1815,0,0,0,5125,5121,2551,2487,87,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,Ilieck,Rated Blitz game,1279,1673,2049,0,0,0,0,1815,0,0,0,0,0,0,16043,16026,7830,7545,668,12,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,Keykodeus,Rated Rapid game,0,0,0,0,0,0,0,0,0,0,0,0,0,0,75,61,30,42,3,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,Solus-Christus,Rated Bullet game,0,1638,1644,1924,0,0,0,1526,0,0,2023,0,0,0,4742,4739,2320,2320,102,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10978,wHiterAven34ece,Rated Blitz game,0,0,764,0,0,0,0,0,0,0,0,0,0,0,132,132,58,74,0,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
10979,eitery,Rated Bullet game,0,1462,1624,1940,0,0,0,0,0,0,1817,0,0,0,2584,2366,1292,1263,29,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0
10980,Leto-II-Atreides,Rated Blitz game,0,1898,2292,0,0,0,0,0,0,0,0,0,0,0,6439,6435,2848,3006,585,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
10981,VictorXI,Rated Blitz game,0,1494,1125,1021,0,0,0,0,0,0,1250,0,0,0,1704,1644,788,848,68,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


<h2> Объеединение получившихся датафреймов в один </h2>

In [19]:
whites_blacks = pd.merge(whites, blacks, left_index=True, right_index=True)
whites_blacks

,Username_x,Mode_x,UltraBullet_x,Bullet_x,Blitz_x,Rapid_x,Classical_x,Correspondence_x,Crazyhouse_x,Chess960_x,Antichess_x,Horde_x,Puzzles_x,Puzzle Storm_x,Puzzle Racer_x,Puzzle Streak_x,games_x,rated_x,wins_x,losses_x,draws_x,bookmarks_x,win_streak_x,lose_streak_x,King of the Hill_x,Three-check_x,Atomic_x,Racing Kings_x,imported games_x,deviation_x,imported game_x,playing_x,Username_y,Mode_y,UltraBullet_y,Bullet_y,Blitz_y,Rapid_y,Classical_y,Correspondence_y,Crazyhouse_y,Chess960_y,Antichess_y,Horde_y,Puzzles_y,Puzzle Storm_y,Puzzle Racer_y,Puzzle Streak_y,games_y,rated_y,wins_y,losses_y,draws_y,bookmarks_y,win_streak_y,lose_streak_y,King of the Hill_y,Three-check_y,Atomic_y,Racing Kings_y,imported games_y,deviation_y,imported game_y,playing_y
0,Apostolis_L,Rated Rapid tournament https://lichess.org/tou...,0,0,1034,1337,0,0,0,0,0,0,1421,15,0,11,385,372,103,274,8,0,0,0,0.0,0.0,0.0,0.0,109.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,Chawteenn,Rated Blitz game,0,1733,1850,0,0,0,0,0,0,0,1432,0,0,0,15065,15044,7171,7023,871,17,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,mathmatical,Rated Blitz game,0,2037,1978,2072,0,2275,0,0,0,0,2415,0,0,0,20147,20073,10115,9364,668,0,0,0,0.0,0.0,0.0,0.0,6.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,Haze0809,Rated Rapid game,0,1027,944,1266,1201,0,0,0,0,0,1486,0,0,0,1322,1294,624,653,45,2,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,PaPanDreo,Rated Bullet game,0,1554,1632,1820,1870,0,0,0,0,0,0,0,0,0,28568,28557,13732,13806,1030,1,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Ngaytaovsmay,Rated Blitz game,0,1541,996,0,0,0,0,0,0,0,1488,0,0,0,362,337,167,184,11,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,bluepenquin,Rated Bullet game,1061,1554,1343,0,0,0,0,0,0,0,2328,0,0,10,4471,3903,2134,2193,144,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,GreySharkfanclubbot,Rated Blitz game,1744,2098,2333,2334,0,0,0,0,0,0,0,0,0,0,12444,12333,5931,5060,1453,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
0,Hughiez,Rated Blitz game,0,681,937,1188,0,0,0,0,0,0,1342,11,0,9,6652,6525,3103,3468,81,3,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


<h2> Объединение получившегося датафрейма с исходными данными </h2>

In [20]:
data = pd.merge(games_stats, whites_blacks, left_index=True, right_index=True)
data = data[~(data.Username_x.isna()) & ~(data.Username_y.isna())].reset_index(drop=True)
data

,Event,Site,Date,Round,White,Black,Result,UTCDate,UTCTime,WhiteElo,BlackElo,WhiteRatingDiff,BlackRatingDiff,ECO,Opening,TimeControl,Termination,WhiteTitle,BlackTitle,Username_x,Mode_x,UltraBullet_x,Bullet_x,Blitz_x,Rapid_x,Classical_x,Correspondence_x,Crazyhouse_x,Chess960_x,Antichess_x,Horde_x,Puzzles_x,Puzzle Storm_x,Puzzle Racer_x,Puzzle Streak_x,games_x,rated_x,wins_x,losses_x,draws_x,bookmarks_x,win_streak_x,lose_streak_x,King of the Hill_x,Three-check_x,Atomic_x,Racing Kings_x,imported games_x,deviation_x,imported game_x,playing_x,Username_y,Mode_y,UltraBullet_y,Bullet_y,Blitz_y,Rapid_y,Classical_y,Correspondence_y,Crazyhouse_y,Chess960_y,Antichess_y,Horde_y,Puzzles_y,Puzzle Storm_y,Puzzle Racer_y,Puzzle Streak_y,games_y,rated_y,wins_y,losses_y,draws_y,bookmarks_y,win_streak_y,lose_streak_y,King of the Hill_y,Three-check_y,Atomic_y,Racing Kings_y,imported games_y,deviation_y,imported game_y,playing_y
0,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,Apostolis_L,Rated Rapid tournament https://lichess.org/tou...,0,0,1034,1337,0,0,0,0,0,0,1421,15,0,11,385,372,103,274,8,0,0,0,0.0,0.0,0.0,0.0,109.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,Chawteenn,Rated Blitz game,0,1733,1850,0,0,0,0,0,0,0,1432,0,0,0,15065,15044,7171,7023,871,17,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,mathmatical,Rated Blitz game,0,2037,1978,2072,0,2275,0,0,0,0,2415,0,0,0,20147,20073,10115,9364,668,0,0,0,0.0,0.0,0.0,0.0,6.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,Haze0809,Rated Rapid game,0,1027,944,1266,1201,0,0,0,0,0,1486,0,0,0,1322,1294,624,653,45,2,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,PaPanDreo,Rated Bullet game,0,1554,1632,1820,1870,0,0,0,0,0,0,0,0,0,28568,28557,13732,13806,1030,1,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,laurent971,Rated Rapid tournament https://lichess.org/tou...,0,0,1177,1382,1434,0,0,0,0,0,1327,0,0,0,16437,16385,5953,9552,932,0,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10751,Rated Rapid tournament https://lichess.org/tou...,https://lichess.org/uManRThA,2023.03.01,-,Apostolis_L,laurent971,1-0,1,55,1326,1328,6.0,-6.0,C62,Ruy Lopez: Steinitz Defense,600+0,Normal,NaN,NaN,Ngaytaovsmay,Rated Blitz game,0,1541,996

<h2> Применение функции для парсинга дополнительных игр </h2>

In [ ]:
import tqdm
from IPython.display import clear_output


d = pd.DataFrame()

df = pd.read_csv('/Users/vsevolodparamonov/Downloads/Датасет шахматисты мнение (1).csv')
games = df[['White', 'Black', 'UTCDate', 'UTCTime']].values


for i in tqdm.tqdm(games):
    pl, op_pl, utc_date, utc_time = i
    df_1 = parsing_add_feat(utc_date, utc_time, pl, op_pl)
    df_2 = parsing_add_feat(utc_date, utc_time, op_pl, pl)
    opa = pd.concat([df_1, df_2], axis=1)

    d = pd.concat([d,opa])

    clear_output(wait=True)  
    display(d)

#### !!! Тут произошел бан со стороны lichess.com :)